In [1]:
import tensorflow as tf
import cv2
from tqdm import tqdm
import time
import urllib, json
import matplotlib.lines as lines

In [2]:
OUTPUT_SHAPE = (8, 8)
RESIZE_VALUE = (256, 256)
IMAGES_PATH = 'images/'
RESIZED_PATH = 'images/resized/'
CLASSES = ['Fish', 'Flower', 'Gravel', 'Sugar']
ORIGINAL_IMAGE_WIDTH = 2100
ORIGINAL_IMAGE_HEIGHT = 1400

In [3]:
def buildBoundingBox(data, resizeValue):

    boxes = []
    for imageid in tqdm(data['Image_Label']):
        imageid, bbox, mask_pixels_count, box_pixels_count = getImageBoundingBox(data, imageid, IMAGES_PATH, ORIGINAL_IMAGE_WIDTH, ORIGINAL_IMAGE_HEIGHT, resizeValue)
        boxes.append(
            {
                'Image_Label': imageid, 
                'XMOY': bbox['XMOY'], 
                'YMOY': bbox['YMOY'], 
                'W': bbox['W'], 
                'H': bbox['H'], 
                'ResizedMaskPixelsCount': mask_pixels_count, 
                'ResizedBoundingBoxPixelsCount': box_pixels_count, 
                'BoxMaskGap': (box_pixels_count - mask_pixels_count) / mask_pixels_count if mask_pixels_count != 0 else 0
            }
        )
    
    bounding_boxes = pd.DataFrame(boxes)
    
    return data.merge(right=bounding_boxes, on='Image_Label', how='inner')

In [4]:
def getImageBoundingBox(data, imageid, image_path, img_width, img_height, size, resize=True):
    
    # Get RLE encoded masks of an image by its imageid and related labels (Flower, Fish...)
    mask_filtered_byId = data[data['Image_Label']==imageid]
    img_mask = mask_filtered_byId['EncodedPixels']

    if np.sum(mask_filtered_byId['EncodedPixelsCount']) != 0:
        
        img_mask = img_mask.values[0]
        
        # Convert one RLE encoded mask into a binary encoded grid
        one_mask = np.zeros((img_height, img_width))
        one_mask = rle_to_mask(img_mask, img_width, img_height)
        
        # Resize Mask size
        if resize:
            one_mask = cv2.resize(one_mask, dsize=size)
        
        one_mask_pixels_count = np.count_nonzero(one_mask == 255)
        
        # Find contours in the binary mask
        contours, _ = cv2.findContours(one_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
        tmp_edges = []
        for contour in contours:
            # Get the bounding box of the contour
            x, y, w, h = cv2.boundingRect(contour)
            tmp_edges.append({'left':x, 'top':y, 'width':w, 'height':h})
        
        edges = pd.DataFrame(tmp_edges)
     
        left = edges['left'].min()
        right = edges[['left', 'width']].sum(axis=1).max()
        top = edges['top'].min()
        bottom = edges[['top', 'height']].sum(axis=1).max()
        
        XMOY = (left + right) / 2
        YMOY = (top + bottom) / 2
        W = right - left
        H = bottom - top
        
        bbox = {'XMOY': XMOY, 'YMOY': YMOY, 'W': W, 'H': H}
        
        return imageid, bbox, one_mask_pixels_count, w * h
    
    else:
        bbox = {'XMOY': 0, 'YMOY': 0, 'W': 0, 'H': 0}
        
        return imageid, bbox, 0, 0
        

In [5]:
# Convertit les coordonnées normalisées de la bbox vers d'autres coordonnées dans le repère d'une case de la grille
def convert_target(target, output_shape=OUTPUT_SHAPE, nb_classes=4):
    
    y_target = np.zeros([output_shape[0], output_shape[1], 1 + 4 + nb_classes])
    lx = 1 / output_shape[1]
    ly = 1 / output_shape[0]
    
    for x, y, w, h, fish, flower, gravel, sugar  in [target]:

        idx_x = int(x//lx)
        idx_y = int(y//ly)
        
        y_target[idx_y, idx_x, 0] = 1 if np.sum([x, y, w, h]) != 0 else 0 # Presence of object
        y_target[idx_y, idx_x, 1] = 2 * (x / lx - (idx_x + 0.5)) # Coordinate x
        y_target[idx_y, idx_x, 2] = 2 * (y / ly - (idx_y + 0.5)) # Coordinate y
        y_target[idx_y, idx_x, 3] = w # Coordinate w
        y_target[idx_y, idx_x, 4] = h # Coordinate h
        y_target[idx_y, idx_x, 5] = fish
        y_target[idx_y, idx_x, 6] = flower
        y_target[idx_y, idx_x, 7] = gravel
        y_target[idx_y, idx_x, 8] = sugar

        
    return y_target.reshape([-1, 1 + 4 + nb_classes])

In [6]:
# Les sorties du modèle sont réinterpretées à l'aide des activateurs qui correspondent au type de la sortie
# Les données sont interpretable par l'humain
def transform_netout(y_pred_raw):

    # outputs sigmoïde [0,1] et notre p_true doit valoir 0 ou 1 
    y_pred_conf = tf.sigmoid(y_pred_raw[..., :1])

    # pour avoir des valeurs comprises entre -1 et 1 avec tanh --> rappel: chaque case de la grille devient un nouveau repère dont l'origine est centrée: le max des abcisses et des ordonnées est 1 et le min -1
    y_pred_xy = (tf.nn.tanh(y_pred_raw[..., 1:3]))
    
    # parce que la largeur et la hauteur de la bbox peuvent valoir 0 au min et 1 au max
    y_pred_wh = tf.sigmoid(y_pred_raw[..., 3:5])

    # pour avoir des probas comprises entre 0 et 1 pour chacune des 4 classes
    y_pred_class = tf.nn.softmax(y_pred_raw[..., 5:])    
    #y_pred_class = tf.sigmoid(y_pred_raw[..., 5:])    

    return tf.concat([y_pred_conf, y_pred_xy, y_pred_wh, y_pred_class], -1)

In [7]:
# def load_image(filepath, y, resize=RESIZE_VALUE):
    
#     img = tf.io.read_file(RESIZED_PATH + filepath)
#     img = tf.image.decode_png(img, channels=3)
#     img = tf.image.resize(img, resize)

#     return img, convert_target(y.numpy().copy(), OUTPUT_SHAPE)

In [8]:
def load_image(filepath, y, resize=RESIZE_VALUE):
    
    im = tf.io.read_file(RESIZED_PATH + filepath)
    im = tf.image.decode_png(im, channels=3)
    im = tf.image.resize(im, resize)
    
    tx_max = resize[1] * tf.nn.relu(y[0] - y[2]/2)
    tx_min = -resize[1] * tf.nn.relu(1 - y[0] - y[2]/2)
    ty_max = resize[0] * tf.nn.relu(y[1] - y[3]/2)
    ty_min = -resize[0] * tf.nn.relu(1 - y[1] - y[3]/2)
    
    tx = np.random.uniform(tx_min, tx_max)
    ty = np.random.uniform(ty_min, ty_max)
    
    im = tf.keras.preprocessing.image.apply_affine_transform(
        im.numpy(), 
        theta=0, 
        tx=ty, ty=tx, 
        shear=0, 
        zx=1, zy=1, 
        row_axis=0, col_axis=1, 
        channel_axis=2, fill_mode='nearest', cval=0.0, order=1
    )
    
    y_new = y.numpy().copy()
    y_new[0] += -tx/resize[1]
    y_new[1] += -ty/resize[0]
    
    return im, convert_target(y_new, OUTPUT_SHAPE)

In [9]:
def class_loss(y_true, y_pred):
    
    y_true_conf = y_true[..., 0] # Vecteur de présence d'un objet
    y_true_class = y_true[..., 5:] # Probabilité conditionelle des vrais objets
    y_pred_class = y_pred[..., 5:] # Probabilité conditionelle des prédictions
    
    # Calcul de la fonction de perte
    class_loss = tf.reduce_sum(y_true_conf * tf.reduce_sum(tf.square(y_true_class - y_pred_class), axis = -1), axis = -1)
    
    return class_loss

In [10]:
def coord_loss(y_true, y_pred):
    
    # Probabilty of object presence
    y_true_conf = y_true[..., 0]
    
    # x and y loss for real object
    y_true_xy = y_true[..., 1:3]
    y_pred_xy = y_pred[..., 1:3]
    xy_loss = tf.reduce_sum(tf.reduce_sum(tf.square(y_true_xy - y_pred_xy), axis =- 1) * y_true_conf, axis = -1)
    
    # w and h loss for real object
    y_true_wh = y_true[..., 3:5]
    y_pred_wh = y_pred[..., 3:5]
    wh_loss = tf.reduce_sum(tf.reduce_sum(tf.square(tf.sqrt(y_true_wh) - tf.sqrt(y_pred_wh)), axis = -1) * y_true_conf, axis = -1)
    
    return xy_loss + wh_loss

In [11]:
lambda_noobj = 0.5

def object_loss(y_true, y_pred):
    
    # x and y loss for real object
    y_true_p = y_true[..., 0]
    y_pred_p = y_pred[..., 0]
    
    return tf.reduce_sum((lambda_noobj + (1 - lambda_noobj) * y_true_p) * tf.square(y_true_p - y_pred_p), axis= -1)

In [12]:
lambda_coord = 5
lambda_object = 1
lambda_class = 1

def global_loss(y_true, y_pred):
    
    # Convert input
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_pred = transform_netout(y_pred)
    
    loss_coordinate = coord_loss(y_true, y_pred)
    loss_object = object_loss(y_true, y_pred)
    loss_class = class_loss(y_true, y_pred)
    
    return lambda_object * loss_object + lambda_coord * loss_coordinate + lambda_class * loss_class

In [13]:
def url_to_image(url):
    
    resp = urllib.request.urlopen(url) 
    img = np.asarray(bytearray(resp.read()), dtype="uint8")
    img = cv2.imdecode(img, -1)
    img = cv2.resize(img, RESIZE_VALUE)
    img = img[..., [2,1,0]]
    
    return tf.keras.applications.efficientnet.preprocess_input(img)

In [14]:
def show_bounding_box(im, bbox, class_pred, ax, normalised=True, color='r'):
    
    # Signification de bbox
    x, y, w, h, fish, flower, gravel, sugar = bbox
    
    # Convertir les cordonées (x,y,w,h) en (x1,x2,y1,y2)
    x1 = x - w / 2
    x2 = x + w / 2
    y1 = y - h / 2
    y2 = y + h / 2
    
    # redimensionner en cas de normalisation
    if normalised:
        x1 = x1 * im.shape[1]
        x2 = x2 * im.shape[1]
        y1 = y1 * im.shape[0]
        y2 = y2 * im.shape[0]

    delta_x = 2
    delta_y = 7
    if y1 - delta_y < 0:
        delta_y_resolved = y1 + delta_y
    else:
        delta_y_resolved = y1 - delta_y

    ax.text(x1 + delta_x, delta_y_resolved, class_pred, fontsize=10, ha='left', va='center', bbox=dict(boxstyle='square', alpha=0.8, facecolor='orange', edgecolor='none'))
    
    # Afficher l'image
    ax.set_title(class_pred)
    ax.imshow(im)
    
    # Afficher la bounding box
    ax.plot([x1, x2, x2, x1, x1],[y1, y1, y2, y2, y1], 'orange')


In [15]:
def generate_yolo_grid(g):
    
    c_x = tf.cast(tf.reshape(tf.tile(tf.range(g), [g]), (1, g, g)), 'float32')
    c_y = tf.transpose(c_x, (0,2,1))
    
    return tf.stack([tf.reshape(c_x, (-1, g*g)), tf.reshape(c_y, (-1, g*g))] , -1)

In [16]:
c_grid = generate_yolo_grid(OUTPUT_SHAPE[0])

def proccess_xy(y_true_raw):

    y_true_conf = y_true_raw[..., :1]
    y_true_xy = ((y_true_raw[..., 1:3] + 1) / 2 + c_grid) / OUTPUT_SHAPE[0]
    y_true_wh = y_true_raw[..., 3:5]
    y_true_class = y_true_raw[..., 5:]
    
    return tf.concat([y_true_conf, y_true_xy, y_true_wh, y_true_class], -1)

Metal device set to: Apple M2 Max


2024-03-13 00:40:58.278330: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-13 00:40:58.278476: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [17]:
def pred_bboxes(y, threshold=0.3):
    
    y_xy = tf.cast(y, tf.float32)
    y_xy = tf.expand_dims(y_xy, axis=0)
    y_xy = proccess_xy(y_xy)[0]
    
    bboxes =  sorted(y_xy.numpy(), key=lambda x: x[0], reverse=True)
    bboxes = np.array(bboxes)
    result = bboxes[bboxes[:,0] > threshold]
    
    if len(result)== 0:
        return bboxes[[0]]
        
    return result


In [18]:
def show_prediction(img, model, ax, threshold=0.8):
    
    pred = model(np.array([img], dtype=np.float32))[0]

    # Interpretation des résultats du modèle
    pred = transform_netout(pred)

    # On sélectionne dans la grille, les préductions (une case) avec l'indice de confiance (PObject) > threshold ou à défaut le plus haut
    bboxes_pred = pred_bboxes(pred, threshold)
    
    class_pred = CLASSES[np.argmax(bboxes_pred[0, 5:])]
    box_prob = round(bboxes_pred[0, 0] * 100, 2)
    class_prob = round(max(bboxes_pred[0, 5:]) * 100, 2)

    plot_title = class_pred + ": " + str(class_prob) + " % - Bbox: " + str(box_prob) + " %"
    
    for bbox in bboxes_pred:
        bbox = bbox[1:]
        show_bounding_box(img/255, bbox, plot_title, ax)
    
    return bboxes_pred

In [19]:
def mean_iou(y_true, y_pred):

    y_true = tf.cast(y_true, tf.float32)
    
    y_pred = tf.cast(y_pred, tf.float32)
    y_pred = transform_netout(y_pred)

    xmoy_true = y_true[..., 1:2]
    ymoy_true = y_true[..., 2:3]
    w_true = y_true[..., 3:4]
    h_true = y_true[..., 4:5]
    
    xmoy_pred = y_pred[..., 1:2]
    ymoy_pred = y_pred[..., 2:3]
    w_pred = y_pred[..., 3:4]
    h_pred = y_pred[..., 4:5]

    x1_true = xmoy_true - w_true / 2
    x2_true = xmoy_true + w_true / 2
    y1_true = ymoy_true - h_true / 2
    y2_true = ymoy_true + h_true / 2

    x1_pred = xmoy_pred - w_pred / 2
    x2_pred = xmoy_pred + w_pred / 2
    y1_pred = ymoy_pred - h_pred / 2
    y2_pred = ymoy_pred + h_pred / 2

    x1 = tf.math.maximum(x1_true, x1_pred)
    y1 = tf.math.maximum(y1_true, y1_pred)
    x2 = tf.math.minimum(x2_true, x2_pred)
    y2 = tf.math.minimum(y2_true, y2_pred)

    intersection = tf.maximum(x2 - x1, 0) * tf.maximum(y2 - y1, 0)
    area_true = w_true * h_true
    area_pred = w_pred * h_pred

    iou = intersection / (area_true + area_pred - intersection + 1e-6)
  
    return iou

In [20]:
# Displays one mask on top of its originate image
def show_ground(image_label, ax, masks, w, h, image_path, hide_axis=False, show_mask=False):

    alpha = 0.2

    filename = image_label.split('_')[0]
    
    image_path = image_path
    img = cv2.imread(image_path + filename)

    if show_mask:
        # Get RLE encoded masks of an image by its image_label and related labels (Flower, Fish...)
        masks_filtered_byId = masks[masks['Image_Label']==image_label]
        img_masks = masks_filtered_byId['EncodedPixels'].tolist()
        img_masks_labels = masks_filtered_byId['Label'].tolist()
    
        # Convert RLE encoded masks into a binary encoded grids
        all_masks = np.zeros((h, w))
        one_mask = np.zeros((h, w))
        mask_origines = []
        for rle_mask in img_masks:
            one_mask = rle_to_mask(rle_mask, w, h)
            mask_origines.append(get_mask_origine(one_mask))
            all_masks += one_mask

    # Displays images and related masks
    if hide_axis:
        ax.axis('off')

    if show_mask:
        # Displays images and related masks
        for origine, label in zip(mask_origines, img_masks_labels):
            ax.annotate(text=label + " 0", xy=origine[0], xytext=(20, -30), xycoords='data', color='yellow', fontsize=10, fontweight='bold', textcoords='offset pixels')
            ax.annotate(text=label + " 1", xy=origine[1], xytext=(-90, 20), xycoords='data', color='yellow', fontsize=10, fontweight='bold', textcoords='offset pixels') 

        cross_size = 75
        cross_0_x = mask_origines[0][0][0]
        cross_0_y = mask_origines[0][0][1]
        cross_1_x = mask_origines[0][1][0]
        cross_1_y = mask_origines[0][1][1]
        
        cross_0_line1 = lines.Line2D([cross_0_x, cross_0_x], [cross_0_y - cross_size, cross_0_y + cross_size], color='y')
        cross_0_line2 = lines.Line2D([cross_0_x - cross_size, cross_0_x + cross_size], [cross_0_y, cross_0_y], color='y')
        cross_1_line1 = lines.Line2D([cross_1_x, cross_1_x], [cross_1_y - cross_size, cross_1_y + cross_size], color='y')
        cross_1_line2 = lines.Line2D([cross_1_x - cross_size, cross_1_x + cross_size], [cross_1_y, cross_1_y], color='y')
    
        # # Add the cross lines to the plot
        ax.add_line(cross_0_line1)
        ax.add_line(cross_0_line2)
        ax.add_line(cross_1_line1)
        ax.add_line(cross_1_line2)
        ###
    
    ax.set_title(image_label)
    
    ax.imshow(img)

    if show_mask:
        ax.imshow(all_masks, alpha=alpha)

In [21]:
# Count non-significant pixels (100% black pixels contained in the black bar)
def countNonSignificantPixels(imageid, image_path, show=False):

    fileName = imageid.split('_')[0]
    img = cv2.imread(image_path + fileName)

    # Images couleur converties en niveaux de gris
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Images en niveaux de gris converties en imahges binaires
    _, binary_img = cv2.threshold(img, 1, 255, cv2.THRESH_BINARY)

    black_pixel_count = img.size - cv2.countNonZero(binary_img)

    if show:
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    
    return black_pixel_count

In [22]:
def getBlackStripesOverlapInfo(imageid, image_path, data, width, height):

    data_filtered_byId = data[data['Image_Label']==imageid]
    
    if np.sum(data_filtered_byId['EncodedPixelsCount']) != 0:
    
        # Get initial image
        fileName = imageid.split('_')[0]
        img = cv2.imread(image_path + fileName)
    
        # Detect black bar
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, blackMask = cv2.threshold(img, 1, 255, cv2.THRESH_BINARY_INV)
        black_pixel_count = img.size - cv2.countNonZero(blackMask)
    
        # Get Bounding Box outline
        bbox = data[data['Image_Label']== imageid][['XMOY','YMOY','W','H']].values[0]
        x1 = bbox[0] - bbox[2]/2
        x2 = bbox[0] + bbox[2]/2
        y1 = bbox[1] - bbox[3]/2
        y2 = bbox[1] + bbox[3]/2
    
        # Get Bounding Box mask
        bboxMask = np.zeros((height, width, 1), dtype=np.uint8)  # Grayscale image, initialized with zeros
        
        x1, x2 = int(x1), int(x2)
        y1, y2 = int(y1), int(y2)
        
        # Fill the inner area of the rectangle with white pixels
        bboxMask[y1:y2, x1:x2] = 255
    
        intersectMask = cv2.bitwise_and(bboxMask, blackMask)
        intersectPixelsCount = np.sum(intersectMask == 255)
        bboxMaskPixelsCount = np.sum(bboxMask == 255)
        
        return img, bboxMask, blackMask, intersectMask, intersectPixelsCount, bboxMaskPixelsCount, round(intersectPixelsCount / bboxMaskPixelsCount, 10)
    
    else:
        return None, None, None, None, 0, 0, 0

In [23]:
def convertEndocedToArray(x):
    if x == -1:
        return 0
    else:
        return np.array(x.split(' ')).astype(int)

In [24]:
def countEncodedPixels(x):
    if np.sum(x) == 0:
        return 0
    else:
        return np.sum(x.reshape(int(np.size(x)/2), 2)[:, 1])

In [25]:
###################################
# EVERYTHING BELOW TO BE REVIEWED #
###################################

In [26]:
# Resize a unique image
def resizeImage(sourcePath, fileName, size):

    img_resized = None
    
    img = cv2.imread(sourcePath + fileName)

    if img is not None:
        img_resized = cv2.resize(img, dsize=size)

    return img_resized

In [27]:
# Resize the content of a source folder and save the result to the target folder
def resizeImagesFromFolder(sourcePath, targetPath, size):

    folderContent = os.listdir(sourcePath)

    filesName = [item for item in folderContent if os.path.isfile(os.path.join(sourcePath, item))]

    for fName in tqdm(filesName):
        resizedImage = resizeImage(sourcePath, fName, size)
        if resizedImage is not None:
            targetFilePath = targetPath + fName
            cv2.imwrite(targetFilePath, resizedImage)
                
    return len(filesName)

In [28]:
# Convert RLE string to a numpy array
def rle_to_mask(rle_string, width, height):
   
    rows, cols = height, width
    
    if rle_string == -1:
        return np.zeros((height, width))
    else:
        rle_numbers = [int(num_string) for num_string in rle_string.split(' ')]
        rle_pairs = np.array(rle_numbers).reshape(-1,2)
        img = np.zeros(rows*cols, dtype=np.uint8)
        for index, length in rle_pairs:
            index -= 1
            img[index:index+length] = 255
        img = img.reshape(cols,rows)
        img = img.T
        return img

In [29]:
# Filter Image pixels with Mask (we get the cloud pixels inside the mask) and return statistics
def filterWithMask(imageid, dataset, w, h, image_path, show=False):
    
    img_id = imageid.split('_')[0]
    img = cv2.imread(image_path + img_id + '.jpg')
    
    dataset = dataset[dataset['ImageId']==imageid]
    rle = dataset['EncodedPixels'].values[0]
    label = dataset['Label'].values[0]

    one_mask = rle_to_mask(rle, w, h)

    indices = np.argwhere(one_mask == 255)
    selected_pixels = img[indices[:, 0], indices[:, 1]]

    mean_value = np.mean(selected_pixels)
    std_value = np.std(selected_pixels)

    if show:
        one_mask = np.expand_dims(one_mask, axis=-1)
        new_img = np.where(one_mask == 255, img, 0)
        
        plt.imshow(new_img)
        plt.title(imageid)
        plt.axis('off')
        plt.show()

    return round(mean_value, 2), round(std_value, 2)

In [30]:
# Returns the pixels that delimit the edges of a region in a mask
def get_mask_origine(mask):
    
    white_pixels = np.array(np.where(mask == 255))
    first_white_pixel = white_pixels[:,0]
    last_white_pixel = white_pixels[:,-1]
    
    return (first_white_pixel[1], first_white_pixel[0]), (last_white_pixel[1], last_white_pixel[0])

In [31]:
def displayBoundingBox(imageid, ax, x, y, w, h):
   
    img_id = imageid.split('_')[0]
    im = cv2.imread('images/small/' + img_id + '.jpg')

    x1 = x - w/2
    x2 = x + w/2
    y1 = y - h/2
    y2 = y + h/2

    #ax.axis('off')
    ax.set_title(imageid)
    ax.imshow(im)
    ax.plot([x1, x2 ,x2, x1, x1],[y1, y1, y2, y2, y1],'yellow')

In [32]:
# def load_image(imageid, filepath, isTrain):
    
#     if isTrain:
#         img_id = tf.strings.split(imageid, sep="_")[0]
#     else:
#         img_id = imageid.split('_')[0]
    
#     img = tf.io.read_file(filepath + img_id + '.jpg')
#     img = tf.image.decode_png(img, channels=3)
    
#     return img

In [33]:
import time

def predict_image(img, model, show=False):

    t0 = time.time()
    x, y, w, h, p = model.predict(np.array([img], dtype=np.float32))[0]
    
    if show:
        
        # Dé-standardiser
        x = x * 525
        y = y * 350
        w = w * 525
        h = h * 350

        x1 = x - w/2
        x2 = x + w/2
        y1 = y - h/2
        y2 = y + h/2
        
        fig = plt.figure(figsize=(8, 8))
        plt.imshow(img)
        plt.plot([x1, x2 ,x2, x1, x1],[y1, y1, y2, y2, y1],'yellow')
        plt.show()

        print(x, y, w, h)
        print("Execution time: ",time.time()-t0,"secondes")
        print("Probability: ", p)
    
    return p

In [34]:
def markDuplicate(data, group_field, count_field):
    g = pd.DataFrame(data.groupby([group_field]).agg({count_field:'count'}).rename({count_field:'Count'}, axis=1))
    g.reset_index(drop=False, inplace=True)
    l = list(g[g['Count'] > 1]['FileId'])
    data['Multiple'] = data['FileId'].apply(lambda fieldid: True if fieldid in l else False )
    return data

In [35]:
def generateMaskFileFromRLE(imageid, size):
    rle_string = df[df['ImageId']==imageid]['EncodedPixels'].tolist()[0]
    mask_img = rle_to_mask(rle_string, 2100, 1400)
    mask_img = cv2.resize(mask_img, dsize=size)
    
    
    # Start Experience #
    img_id = imageid.split('_')[0]
    img = cv2.imread('images/square/' + img_id + '.jpg')
    mask_img = np.expand_dims(mask_img, axis=-1)
    mask_img = cv2.bitwise_and(img, img, mask=mask_img)
    # End experience #
    
    #new_path = 'images/small/masks/' + imageid + '_mask.jpg'
    new_path = 'images/square/' + imageid + '_mask.jpg'
    if not(os.path.exists(new_path)):
        cv2.imwrite(new_path, mask_img)
    
    return mask_img

In [36]:
import tensorflow as tf

def loadImages(img_id, mask_id):
    
    #img_filepath = 'images/small/' + img_id + '.jpg'
    #mask_filepath = 'images/small/masks/' + mask_id + '_mask.jpg'
    
    img_filepath = 'images/square/' + img_id + '.jpg'
    mask_filepath = 'images/square/' + mask_id + '_mask.jpg'

    img = tf.io.read_file(img_filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [350, 525])

    mask = tf.io.read_file(mask_filepath)
    mask = tf.io.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, [350, 525], method = 'nearest')
   
    # Normilalize
    img = tf.cast(img, tf.float32)/255
    mask = tf.cast(mask, tf.float32)/255

    return img, mask

In [37]:
import tensorflow as tf

def loadImages_square(img_id, mask_id):
    
    img_filepath = 'images/square/' + img_id + '.jpg'
    mask_filepath = 'images/square/' + mask_id + '_mask.jpg'

    img = tf.io.read_file(img_filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [128, 128])

    mask = tf.io.read_file(mask_filepath)
    mask = tf.io.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, [128, 128], method = 'nearest')
   
    # Normilalize
    img = tf.cast(img, tf.float32)/255
    mask = tf.cast(mask, tf.float32)/255

    return img, mask